In [1]:
import logging
import sys
from pathlib import Path

project_root = Path.cwd().parent
sys.path.insert(0, str(project_root))

logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(name)s - %(levelname)s - %(message)s")

In [2]:
from tplexity.retriever.retriever_service import RetrieverService

service = RetrieverService()

2025-11-15 22:43:25,091 - tplexity.retriever.retriever_service - INFO - 🔄 [retriever_service] Инициализация гибридного поисковика
2025-11-15 22:43:25,091 - tplexity.retriever.vector_search - INFO - 🔄 [retriever][vector_search] Инициализация клиента Qdrant
2025-11-15 22:43:25,478 - httpx - INFO - HTTP Request: GET https://8b725c10-321f-428c-8fc8-00160f7dff00.eu-central-1-0.aws.cloud.qdrant.io:6333 "HTTP/1.1 200 OK"
2025-11-15 22:43:25,479 - tplexity.retriever.vector_search - INFO - ✅ [retriever][vector_search] Клиент Qdrant инициализирован: 8b725c10-321f-428c-8fc8-00160f7dff00.eu-central-1-0.aws.cloud.qdrant.io:6333
2025-11-15 22:43:25,480 - tplexity.retriever.dense_embedding - INFO - 🔄 [retriever][dense_embedding] Инициализация модели: jinaai/jina-embeddings-v3
2025-11-15 22:43:25,539 - sentence_transformers.SentenceTransformer - INFO - Use pytorch device_name: mps
2025-11-15 22:43:25,539 - sentence_transformers.SentenceTransformer - INFO - Load pretrained SentenceTransformer: jinaai/j

In [3]:
count_result = service.vector_search.client.count(collection_name=service.vector_search.collection_name)
print(f"Количество документов в коллекции '{service.vector_search.collection_name}': {count_result.count}")

2025-11-15 22:44:29,252 - httpx - INFO - HTTP Request: POST https://8b725c10-321f-428c-8fc8-00160f7dff00.eu-central-1-0.aws.cloud.qdrant.io:6333/collections/documents/points/count "HTTP/1.1 200 OK"


Количество документов в коллекции 'documents': 5


In [4]:
documents = [
    "Банковский вклад - это денежные средства, размещенные в банке на определенный срок под проценты. Вклады бывают срочные и до востребования.",
    "Кредит - это заем денежных средств, предоставляемый банком клиенту на определенных условиях. Кредиты бывают потребительские, ипотечные и автокредиты.",
    "Для открытия счета в банке необходимо предоставить паспорт, заполнить заявление и внести первоначальный взнос. Счет можно открыть онлайн или в отделении банка.",
    "Дебетовая карта позволяет расплачиваться собственными средствами, хранящимися на счете. Кредитная карта позволяет тратить заемные средства банка.",
    "Ипотечный кредит выдается на покупку недвижимости. Обычно требуется первоначальный взнос от 10% до 30% от стоимости недвижимости.",
]

metadatas = [
    {"type": "deposit", "category": "banking"},
    {"type": "credit", "category": "banking"},
    {"type": "account", "category": "banking"},
    {"type": "card", "category": "banking"},
    {"type": "mortgage", "category": "credit"},
]

service.add_documents(documents, metadatas=metadatas)
print(f"Добавлено документов: {len(documents)}")

2025-11-15 22:44:31,276 - tplexity.retriever.retriever_service - INFO - 🔄 [retriever_service] Добавление 5 новых документов


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

2025-11-15 22:45:21,791 - tplexity.retriever.vector_search - ERROR - ❌ [retriever][vector_search] Ошибка при добавлении документов в Qdrant: The read operation timed out
2025-11-15 22:45:21,795 - tplexity.retriever.retriever_service - ERROR - ❌ [retriever_service] Ошибка при добавлении документов в Qdrant: The read operation timed out


ResponseHandlingException: The read operation timed out

In [ ]:
results = service.search("Что такое банковский вклад?", top_k=10, top_n=5, use_rerank=False)
print(f"Найдено результатов: {len(results)}")
for doc_id, score, text, metadata in results:
    print(f"Doc ID: {doc_id}, Score: {score:.3f}")
    print(f"Text: {text[:100]}...")
    print(f"Metadata: {metadata}")
    print()

In [ ]:
results = service.search("Как открыть счет?", top_k=5, top_n=3, use_rerank=False)
print(f"Найдено результатов: {len(results)}")
for doc_id, score, text, metadata in results:
    print(f"Doc ID: {doc_id}, Score: {score:.3f}")
    print(f"Text: {text[:100]}...")
    print(f"Metadata: {metadata}")
    print()

In [ ]:
results = service.search("кредит", top_k=3, top_n=2, use_rerank=False)
print(f"Найдено результатов (без rerank): {len(results)}")
for doc_id, score, text, metadata in results:
    print(f"Doc ID: {doc_id}, Score: {score:.3f}")
    print(f"Text: {text[:100]}...")
    print(f"Metadata: {metadata}")
    print()

In [ ]:
results = service.search("ипотека", top_k=10, top_n=1)
if results:
    doc_id_to_delete = results[0][0]
    print(f"Найден документ для удаления: {doc_id_to_delete}")
    print(f"Text: {results[0][2][:100]}...")
else:
    doc_id_to_delete = None
    print("Не найдено документов")

In [ ]:
if doc_id_to_delete:
    service.delete_documents([doc_id_to_delete])
    print(f"Удален документ: {doc_id_to_delete}")

    results = service.search("ипотека", top_k=10, top_n=5)
    print(f"\nПоиск после удаления: найдено {len(results)} результатов")

In [ ]:
service.add_documents(["Дополнительный документ о банковских услугах"], metadatas=[{"source": "additional"}])
print("Добавлен дополнительный документ")

In [ ]:
results = service.search("банковские услуги", top_k=10, top_n=5)
print(f"Найдено результатов после добавления: {len(results)}")
for doc_id, score, text, metadata in results:
    print(f"Doc ID: {doc_id}, Score: {score:.3f}")
    print(f"Text: {text[:80]}...")
    print(f"Metadata: {metadata}")
    print()

In [ ]:
service.delete_all_documents()
print("Все документы удалены")

In [ ]:
results = service.search("банковский вклад", top_k=10, top_n=5)
print(f"Найдено результатов после удаления всех: {len(results)}")
print(f"Results: {results}")

In [ ]:
# Тест получения всех документов через метод сервиса
all_documents = service.get_all_documents()
print(f"Всего документов: {len(all_documents)}\n")
for i, (doc_id, text, metadata) in enumerate(all_documents, 1):
    print(f"Документ {i}:")
    print(f"  ID: {doc_id}")
    print(f"  Текст: {text}")
    if metadata:
        print(f"  Метаданные: {metadata}")
    print()

In [ ]:
# Тест получения одного документа по ID
# Получаем результаты поиска для получения ID
search_results_for_single = service.search("банковский вклад", top_k=1, top_n=1, use_rerank=False)
if search_results_for_single:
    single_doc_id = [search_results_for_single[0][0]]  # Берем ID первого документа
    print(f"Получение одного документа по ID: {single_doc_id[0]}\n")

    single_doc = service.get_documents(single_doc_id)
    if single_doc:
        doc_id, text, metadata = single_doc[0]
        print("Документ:")
        print(f"  ID: {doc_id}")
        print(f"  Текст: {text}")
        if metadata:
            print(f"  Метаданные: {metadata}")
    else:
        print("Документ не найден")
else:
    print("Нет документов для тестирования")

In [ ]:
# Тест гибридного поиска с MMR (Maximum Marginal Relevance)
# MMR добавлен в dense prefetch для увеличения разнообразия результатов
print("Тест гибридного поиска с MMR:\n")
results = service.search("кредит банк", top_k=10, top_n=5, use_rerank=True)
print(f"Найдено результатов: {len(results)}")
print("\nРезультаты (с reranking и MMR):")
for i, (doc_id, score, text, metadata) in enumerate(results, 1):
    print(f"{i}. Doc ID: {doc_id}, Score: {score:.4f}")
    print(f"   Text: {text[:80]}...")
    if metadata:
        print(f"   Metadata: {metadata}")
    print()

In [ ]:
# Тест получения всех документов после добавления новых
print("Добавляем новые документы для теста...\n")
new_docs = [
    "Инвестиции - это вложение денежных средств с целью получения дохода в будущем.",
    "Страхование защищает от финансовых потерь в случае наступления страхового случая.",
]
new_metadatas = [{"type": "investment", "category": "finance"}, {"type": "insurance", "category": "finance"}]

service.add_documents(new_docs, metadatas=new_metadatas)
print(f"Добавлено {len(new_docs)} новых документов\n")

# Получаем все документы
all_docs = service.get_all_documents()
print(f"Всего документов в базе: {len(all_docs)}\n")
print("Последние добавленные документы:")
for doc_id, text, metadata in all_docs[-len(new_docs) :]:
    print(f"  ID: {doc_id}")
    print(f"  Text: {text}")
    print(f"  Metadata: {metadata}")
    print()

In [ ]:
# Тест получения документов по несуществующим ID
print("Тест получения документов по несуществующим ID:\n")
non_existent_ids = ["non-existent-id-1", "non-existent-id-2"]
retrieved = service.get_documents(non_existent_ids)
print(f"Запрошено документов: {len(non_existent_ids)}")
print(f"Получено документов: {len(retrieved)}")
print("(Qdrant не возвращает документы с несуществующими ID, поэтому список будет пустым)")

In [ ]:
# Тест получения документов по ID
# Сначала получим ID документов из поиска
search_results = service.search("банковский вклад", top_k=3, top_n=3, use_rerank=False)
if search_results:
    doc_ids = [doc_id for doc_id, _, _, _ in search_results]
    print(f"Получение документов по ID: {doc_ids}\n")

    # Получаем документы по ID
    retrieved_docs = service.get_documents(doc_ids)
    print(f"Получено документов: {len(retrieved_docs)}\n")

    for i, (doc_id, text, metadata) in enumerate(retrieved_docs, 1):
        print(f"Документ {i}:")
        print(f"  ID: {doc_id}")
        print(f"  Текст: {text[:100]}...")
        if metadata:
            print(f"  Метаданные: {metadata}")
        print()
else:
    print("Нет документов для тестирования")